In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Libraries

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D,Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam, RMSprop
import optuna

# Training data

In [ ]:
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
train_data.shape

In [ ]:
train_data.head()

In [ ]:
X = train_data.drop('label',axis=1)
y = train_data[['label']]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
y.shape

In [ ]:
y = to_categorical(y)

In [ ]:
y.shape

In [ ]:
y[0]

In [ ]:
y[3]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=100)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
X.max().unique()

In [ ]:
X.min().unique()

# Scaling

In [ ]:
X_train=X_train/255
X_test=X_test/255

In [ ]:
X_train

In [ ]:
X_test

# Reshaping

In [ ]:

X_train = X_train.values.reshape( -1 , 28 , 28 , 1)
X_test = X_test.values.reshape( -1 , 28 , 28 , 1)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
plt.imshow(X_train[10],cmap='gray')

In [ ]:
plt.imshow(X_train[100],cmap='gray')

# Model

In [ ]:
#Sequential API
model = Sequential()
#Convolutional layer
model.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(28,28,1),activation='relu'))
# Pooling layer
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())    
model.add(Dense(128,activation='relu'))
#output layer multiclass hence softmax layer
model.add(Dense(10,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='value_loss',patience=1)

In [ ]:
model.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test),callbacks=[early_stop])

# Evaluation

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics[["loss","val_loss"]].plot()

In [ ]:
metrics[["accuracy","val_accuracy"]].plot()

In [ ]:
model.metrics_names

In [ ]:
model.evaluate(X_test,y_test,verbose=0)

In [ ]:
preds = model.predict(X_test)

In [ ]:
preds

In [ ]:
preds = preds.astype('int32')

In [ ]:
preds

In [ ]:
y_test

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
model_score = model.evaluate(X_test, y_test, verbose=0)
print('validation loss:', model_score[0])
print('validation accuracy:', model_score[1])

# Test data

In [ ]:
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
test_data.shape

In [ ]:
test_data = test_data / 255

In [ ]:
test_data = test_data.values.reshape( -1 , 28 , 28 , 1)

In [ ]:
test_data.shape

In [ ]:
plt.imshow(test_data[0],cmap='gray')

In [ ]:
plt.imshow(test_data[10],cmap='gray')

In [ ]:
test_preds = model.predict(test_data)

In [ ]:
test_preds

In [ ]:
test_preds.shape

#### preds contains the predicted labels of the digits .¶
#### argmax(axis = 1) is converting the one hot encoder back to the labels along the row axis

In [ ]:
preds = np.argmax(test_preds , axis = 1)
test_image_id = range( 1 , len(preds)+1 )

In [ ]:
preds

In [ ]:
len(test_image_id)


In [ ]:
submit_df = {"Id" : test_image_id , "Label" : preds }
submit_df = pd.DataFrame(submit_df)

In [ ]:
submit_df.head()

In [ ]:
submit_df.to_csv('submission.csv',index=False) 

#### Reading link
#### https://github.com/deadskull7/MNIST-digit-recognition-and-classification-using-CNN-with-Keras-99.70/blob/master/MNIST%20digit%20recognition%20%5B0.9932%5D.ipynb

# CNN Model2

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu',input_shape=(28,28,1),)) # 32 filters and each filter is 3 by 3
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,epochs=10,batch_size=128,validation_data=(X_test,y_test))

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics[["loss","val_loss"]].plot()

In [ ]:
metrics[["accuracy","val_accuracy"]].plot()

In [ ]:
preds = model.predict(X_test)
preds=preds.astype('int32')

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
test_preds = model.predict(test_data)

In [ ]:
test_preds

In [ ]:
test_preds.shape

In [ ]:
preds = np.argmax(test_preds , axis = 1)
test_image_id = range( 1 , len(preds)+1 )
preds

In [ ]:
len(test_image_id)

In [ ]:
submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
submission.head()

In [ ]:
submit_df = {"ImageId" : test_image_id , "Label" : preds }
submit_df = pd.DataFrame(submit_df)

In [ ]:
submit_df

In [ ]:
submit_df.to_csv('submission.csv',index=False) # 0.98375

# CNN Model3 

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3), activation='relu',input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(10,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,epochs=10,batch_size=128,validation_data=(X_test,y_test))

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics[["loss","val_loss"]].plot() # by second epoch validation loss came down then going up and down and up which is sign of overfitting

In [ ]:
metrics[["accuracy","val_accuracy"]].plot() # similarly accuracy increased then by 5th epoch the accuracy increased then going down and up

In [ ]:
preds = model.predict(X_test)
preds=preds.astype('int32')

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
test_preds = model.predict(test_data)
test_preds

In [ ]:
test_preds.shape

In [ ]:
preds = np.argmax(test_preds , axis = 1)
test_image_id = range( 1 , len(preds)+1 )
preds

In [ ]:
len(test_image_id)

In [ ]:
submit_df = {"ImageId" : test_image_id , "Label" : preds }
submit_df = pd.DataFrame(submit_df)

In [ ]:
submit_df

In [ ]:
submit_df.to_csv('submission.csv',index=False) # 0.98642 stacking multiple convolutional layers didnot increase the accuracy tremendously.

# Hyperparameter tuning with Optuna

In [ ]:
# objective function

def objective(trial):

    # Keras Sequential model.
    model = Sequential()

    # Convolutional layers.

    # the number of convolution layers
    n_conv_layers = trial.suggest_int('n_conv_layers', 1, 3)

    for i in range(n_conv_layers):
        
        # sample different filters, kernels, stride for each convolutional layer

        model.add(Conv2D(
            filters=trial.suggest_categorical('filters_{}'.format(i), [16, 32, 64]),
            kernel_size=trial.suggest_categorical('kernel_size{}'.format(i), [3, 5]),
            strides=trial.suggest_categorical('strides{}'.format(i), [1, 2]),
            activation='relu',
            padding='same',
        ))

    # Max Pooling
    model.add(MaxPool2D(pool_size=2, strides=2))

    # Flattening for the dense layer
    model.add(Flatten())

    # fully-connected Dense layers.
    # The number of layers is a hyper-parameter we want to optimize.

    n_dense_layers = trial.suggest_int('n_dense_layers', 1, 3)

    for i in range(n_dense_layers):

        # we want to optimize the number of nodes (neurons) and the activation function.
        model.add(Dense(
            units=trial.suggest_int('units{}'.format(i), 5, 512),
            activation=trial.suggest_categorical(
                'activation{}'.format(i), ['relu', 'tanh']),
        ))

    # the softmax layer for classification
    model.add(Dense(10, activation='softmax'))

    # the optimizer 
    optimizer_name = trial.suggest_categorical(
        'optimizer_name', ['Adam', 'RMSprop'])

    if optimizer_name == 'Adam':
        optimizer = Adam(learning_rate=trial.suggest_float('learning_rate',  1e-6, 1e-2))
    else:
        optimizer = RMSprop(
            learning_rate=trial.suggest_float('learning_rate',  1e-6, 1e-2),
            momentum=trial.suggest_float('momentum',  0.1, 0.9),
        )

    # In Keras we need to compile the model so it can be trained.
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # model training
    history = model.fit(
        x=X_train,
        y=y_train,
        epochs=5,
        batch_size=128,
        validation_split=0.1,
    )

    # Get the classification accuracy on the validation-set
    # after the last training-epoch.
    accuracy = history.history['val_accuracy'][-1]

    return accuracy

In [ ]:
study = optuna.create_study(
    direction='maximize'
)

study.optimize(objective, n_trials=10)

In [ ]:
study.best_params

In [ ]:
study.best_value

In [ ]:
#Sequential API
model = Sequential()
#Convolutional layer
model.add(Conv2D(filters=16,kernel_size=(3,3),input_shape=(28,28,1),activation='relu'))
# Pooling layer
model.add(MaxPool2D(pool_size=(2,2)))
#Convolutional layer
model.add(Conv2D(filters=32,kernel_size=(5,5),activation='relu'))
# Pooling layer
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())    
model.add(Dense(242,activation='relu'))
model.add(Dense(27,activation='tanh'))
#output layer multiclass hence softmax layer
model.add(Dense(10,activation='softmax'))

In [ ]:
opt = Adam(learning_rate=0.004618995519239905)
model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,epochs=10,batch_size=128,validation_data=(X_test,y_test))

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics[["loss","val_loss"]].plot() # by second epoch validation loss came down then going up and down and up which is sign of overfitting

In [ ]:
metrics[["accuracy","val_accuracy"]].plot() # similarly accuracy increased then by 5th epoch the accuracy increased then going down and up

In [ ]:
preds = model.predict(X_test)
preds

In [ ]:
preds.shape

In [ ]:
preds = np.argmax(preds , axis = 1)
preds

In [ ]:
preds.shape

In [ ]:
y_test = np.argmax(y_test , axis = 1)
y_test

In [ ]:
print(classification_report(y_test, preds))

In [ ]:
test_preds = model.predict(test_data)
test_preds

In [ ]:
test_preds.shape

In [ ]:
preds = np.argmax(test_preds , axis = 1)
test_image_id = range( 1 , len(preds)+1 )
preds

In [ ]:
len(test_image_id)

In [ ]:
submit_df = {"ImageId" : test_image_id , "Label" : preds }
submit_df = pd.DataFrame(submit_df)

In [ ]:
submit_df

In [ ]:
submit_df.to_csv('submission.csv',index=False) # 0.98607

#### Reading refrence .... Hyperoptimization Udemy